### Pade objective builder function

In [1]:
using Symbolics
using LinearAlgebra
using DynamicPolynomials

In [2]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

Lindblad master equation is given by
\begin{align}\label{eq:L1}
    \frac{d}{dt}{\rho}(t) = \mathcal{L}\rho(t),
\end{align}

where Lindbladian right hand side operator is given as
\begin{align}\label{eq:L2}
    \mathcal{L} [\rho] = \frac{i}{\hbar}[H, \rho]+\sum_{j=1}^{\ell}\left[2 A_j \rho A_j^\dagger - \left\{ A_j^\dagger A_j, \rho \right\} \right],
\end{align}

In [3]:
function lindblad_rhs(rho, H, A)
    """
    Right hand side of the Lindblad master equation
    """
    return -1im * (H * rho - rho * H) + A * rho * A' - (A' * A  * rho + rho * A' * A) / 2
    
end

lindblad_rhs (generic function with 1 method)

Terms for the objective for minimization:
\begin{align}
{\rho_{(i)} - \rho_{(0)} - \Delta t \mathcal{L} \left[\frac{{\rho}_{(i)} + {\rho}_{(i-1)}}{2} \right]} \rightarrow \operatorname{min}
\end{align}

In [4]:
function pade_obj(ρ, H, A)
    obj = frobenius_norm2(lindblad_rhs(ρ[:, :, 1], H, A))
    for i in 2:size(ρ,3)
        obj += frobenius_norm2(ρ[:, :, i] - ρ[:, :, 1] - lindblad_rhs(ρ[:, :, i], H, A))
    end
    obj2 = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj2
end

pade_obj (generic function with 1 method)

Consider test example

In [5]:
@polyvar x[1:4]
H = [ 1.0 * x[1]              x[3] + im * x[4]
      x[3] - im * x[4]        x[2]             ]

@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

ρ1 = [.5 -im
     +im  .5]
ρ2 = [.6 0
     0  .4]
ρ3 = [.7 0
     0  .3]
ρ4 = [.9 0
     0  .1]

ρ = [ρ1;;; ρ2;;; ρ3;;; ρ4]

obj = pade_obj(ρ, H, A);

In [6]:
using TSSOS

opt,sol,data = tssos_first(obj, variables(obj));

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[22, 19, 4]
[1, 1, 2]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 22.
Assembling the SDP...
There are 267 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 267             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - trie

In [7]:
opt,sol,data = tssos_higher!(data);

Starting to compute the block structure...
No higher TSSOS hierarchy!


In [8]:
opt,sol,data = tssos_higher!(data);

Starting to compute the block structure...
No higher TSSOS hierarchy!
